https://chatgpt.com/share/67fff13b-8be8-8000-a638-12c7d433575f

https://chatgpt.com/share/67fff152-a98c-8000-a4bb-10740207e202

In [1]:
! pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 80.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 3.3 MB/s eta 0:00:00


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model

# Step 1: Load and preprocess MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0
x_train = x_train.reshape((x_train.shape[0], 28, 28, 1))
x_test = x_test.reshape((x_test.shape[0], 28, 28, 1))

# Step 2: Define the Autoencoder Model
input_img = layers.Input(shape=(28, 28, 1))

# Encoder
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
encoded = layers.MaxPooling2D((2, 2), padding='same')(x)

# Decoder
x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder Model
autoencoder = Model(input_img, decoded)

# Encoder Model (For feature extraction)
encoder = Model(input_img, encoded)

# Compile and train the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(x_train, x_train, epochs=5, batch_size=32, validation_data=(x_test, x_test))

# Step 3: Extract Features using the Encoder
encoded_train = encoder.predict(x_train)
encoded_test = encoder.predict(x_test)

# Flatten the encoded features for the classifier
encoded_train = encoded_train.reshape((encoded_train.shape[0], -1))
encoded_test = encoded_test.reshape((encoded_test.shape[0], -1))

# Step 4: Train a Classifier on the Encoded Features
classifier = models.Sequential()
classifier.add(layers.Dense(64, activation='relu', input_dim=encoded_train.shape[1]))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dense(256, activation='relu'))
classifier.add(layers.Dense(10, activation='softmax'))  # 10 classes for MNIST

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(encoded_train, y_train, epochs=10, batch_size=128, validation_data=(encoded_test, y_test))

# Step 5: Evaluate the Classifier
test_loss, test_acc = classifier.evaluate(encoded_test, y_test)
print(f'Test Accuracy: {test_acc}')


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.1094 - val_loss: 0.0675
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 16s 4ms/step - loss: 0.0675 - val_loss: 0.0653
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - loss: 0.0654 - val_loss: 0.0642
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - loss: 0.0645 - val_loss: 0.0637
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 4ms/step - loss: 0.0638 - val_loss: 0.0630
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.8515 - loss: 0.4810 - val_accuracy: 0.9597 - val_loss: 0.1405
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9634 - loss: 0.1233 - val_accuracy: 0.9648 - val_loss: 0.1048
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9724 - loss: 0.0897 - val_accuracy: 0.9732 - val_loss: 0.0857
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9784 - loss: 0.0724 - val_accuracy: 0.9769 - val_loss: 0.0710
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9823 - loss: 0.0576 - val_accuracy: 0.9761 - val_loss: 0.0712
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9832 - loss: 0.0559 - val_accuracy: 0.9834 - val_loss: 0.0503
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9851 - loss: 0.0482 - val_accuracy: 0.9840 - val_loss: 0.0517
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9871 - loss: 0.0411 - val_accuracy: 0.

In [2]:
# Step 4: Train a Classifier on the Encoded Features
classifier = models.Sequential()
classifier.add(layers.Dense(64, activation='relu', input_dim=encoded_train.shape[1]))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dense(256, activation='relu'))
classifier.add(layers.Dense(10, activation='softmax'))  # 10 classes for MNIST

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.fit(encoded_train, y_train, epochs=10, batch_size=128, validation_data=(encoded_test, y_test))

# Step 5: Evaluate the Classifier
test_loss, test_acc = classifier.evaluate(encoded_test, y_test)
print(f'Test Accuracy: {test_acc}')


Epoch 1/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - accuracy: 0.8407 - loss: 0.5127 - val_accuracy: 0.9522 - val_loss: 0.1523
Epoch 2/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9635 - loss: 0.1161 - val_accuracy: 0.9628 - val_loss: 0.1160
Epoch 3/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9729 - loss: 0.0872 - val_accuracy: 0.9760 - val_loss: 0.0778
Epoch 4/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.9777 - loss: 0.0688 - val_accuracy: 0.9557 - val_loss: 0.1325
Epoch 5/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - accuracy: 0.9788 - loss: 0.0639 - val_accuracy: 0.9766 - val_loss: 0.0794
Epoch 6/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9833 - loss: 0.0520 - val_accuracy: 0.9806 - val_loss: 0.0600
Epoch 7/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9842 - loss: 0.0481 - val_accuracy: 0.9810 - val_loss: 0.0618
Epoch 8/10
469/469 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.9832 - loss: 0.0507 - val_accuracy: 0.

In [4]:
# Step 4: Train a Classifier on the Encoded Features
classifier = models.Sequential()
classifier.add(layers.Dense(256, activation='relu', input_dim=encoded_train.shape[1]))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dense(256, activation='relu'))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dense(256, activation='relu'))
classifier.add(layers.Dense(128, activation='relu'))
classifier.add(layers.Dense(10, activation='softmax'))  # 10 classes for MNIST

classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
classifier.summary()
classifier.fit(encoded_train, y_train, epochs=5, batch_size=32, validation_data=(encoded_test, y_test))

# Step 5: Evaluate the Classifier
test_loss, test_acc = classifier.evaluate(encoded_test, y_test)
print(f'Test Accuracy: {test_acc}')


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_11 (Dense)                │ (None, 256)            │       803,072 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 969,098 (3.70 MB)

 Trainable params: 969,098 (3.70 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 11s 4ms/step - accuracy: 0.8395 - loss: 0.4793 - val_accuracy: 0.9601 - val_loss: 0.1359
Epoch 2/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9582 - loss: 0.1459 - val_accuracy: 0.9693 - val_loss: 0.1012
Epoch 3/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.9639 - loss: 0.1224 - val_accuracy: 0.9709 - val_loss: 0.1038
Epoch 4/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 7s 4ms/step - accuracy: 0.9711 - loss: 0.1031 - val_accuracy: 0.9779 - val_loss: 0.0765
Epoch 5/5
1875/1875 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.9763 - loss: 0.0839 - val_accuracy: 0.9737 - val_loss: 0.0907
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9691 - loss: 0.1041
Test Accuracy: 0.9736999869346619
